O objetivo deste projeto é prever a **quantidade de consumo mensal** com base em variáveis 
disponíveis no dataset `DasaMatHosp.xlsx`.

A variável alvo é: **quantidade_consumo_mensal**.

Esse tipo de previsão é útil para **gestão de recursos hospitalares**, pois permite antecipar 
necessidades de consumo e otimizar logística de suprimentos.



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

# Carregar os dados
df = pd.read_excel("DasaMatHosp.xlsx")

# Visualizar primeiras linhas
df.head()


ModuleNotFoundError: No module named 'seaborn'

Antes de aplicar regressão linear, precisamos verificar alguns pressupostos:

- **Linearidade**: a relação entre variáveis independentes e a variável alvo deve ser aproximadamente linear.
- **Homoscedasticidade**: os resíduos devem ter variância constante.
- **Normalidade dos resíduos**: os erros devem seguir distribuição aproximadamente normal.
- **Ausência de multicolinearidade**: as variáveis independentes não podem ser altamente correlacionadas entre si.

Essas verificações serão feitas após o ajuste dos modelos.


Vamos começar com um modelo simples, relacionando apenas **uma variável independente** 
com a quantidade de consumo mensal.  

Para este exemplo, utilizaremos a variável **tempo_fidelidade** (ou outra que exista no dataset).


In [ ]:
# Definindo variáveis
X = df[['tempo_fidelidade']]   # variável independente de exemplo
y = df['quantidade_consumo_mensal']  # variável alvo

# Treinando modelo simples
modelo_simples = LinearRegression()
modelo_simples.fit(X, y)

print("Coeficiente (inclinação):", modelo_simples.coef_[0])
print("Intercepto:", modelo_simples.intercept_)


- **Coeficiente (inclinação):** representa a variação média no consumo mensal para cada unidade 
adicional de tempo de fidelidade.
- **Intercepto:** é o valor esperado do consumo mensal quando o tempo de fidelidade é zero.


In [ ]:
# Predições
y_pred_simples = modelo_simples.predict(X)

# Métricas
r2 = r2_score(y, y_pred_simples)
rmse = np.sqrt(mean_squared_error(y, y_pred_simples))

print("R²:", r2)
print("RMSE:", rmse)

# Visualização
plt.scatter(X, y, alpha=0.6, label="Dados reais")
plt.plot(X, y_pred_simples, color="red", label="Regressão Linear")
plt.xlabel("Tempo de Fidelidade")
plt.ylabel("Consumo Mensal")
plt.legend()
plt.show()


Agora expandiremos o modelo para múltiplas variáveis independentes.  
Selecionaremos pelo menos **três variáveis relevantes** do dataset.


In [ ]:
# Seleção de variáveis (ajuste para colunas existentes no seu dataset)
X_multi = df[['tempo_fidelidade', 'numero_moradores', 'renda']]
y = df['quantidade_consumo_mensal']

# Modelo de regressão múltipla
modelo_multi = LinearRegression()
modelo_multi.fit(X_multi, y)

print("Coeficientes:", modelo_multi.coef_)
print("Intercepto:", modelo_multi.intercept_)


Cada coeficiente mostra o **impacto médio** de uma variável independente no consumo mensal:

- **Renda:** espera-se impacto positivo (maior renda → maior consumo).
- **Número de moradores:** espera-se impacto positivo (mais pessoas → mais consumo).
- **Tempo de fidelidade:** pode aumentar consumo, mas precisa ser validado.

Caso algum coeficiente apresente sinal inesperado, isso pode indicar:
- Correlação espúria.
- Multicolinearidade.
- Efeitos não-lineares.


In [ ]:
# Ajustando modelo com statsmodels para diagnóstico
X_multi_sm = sm.add_constant(X_multi)
modelo_sm = sm.OLS(y, X_multi_sm).fit()
print(modelo_sm.summary())

# Resíduos
residuos = y - modelo_multi.predict(X_multi)

# Distribuição dos resíduos
sns.histplot(residuos, kde=True)
plt.title("Distribuição dos Resíduos")
plt.show()

# Resíduos vs Previsões
sns.scatterplot(x=modelo_multi.predict(X_multi), y=residuos)
plt.axhline(0, color='red', linestyle='--')
plt.title("Resíduos vs Previsões")
plt.show()


In [ ]:

kf = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(modelo_multi, X_multi, y, cv=kf, scoring="r2")

print("R² por fold:", scores)
print("Média R²:", scores.mean())


O modelo pode ser aplicado no **planejamento hospitalar**, ajudando em:

- **Gestão de estoque:** prever consumo de insumos hospitalares mensalmente.
- **Eficiência logística:** reduzir faltas e desperdícios.
- **Suporte à decisão:** embasar políticas de compras e contratos.

### Benefícios
- Antecipação de demanda.
- Otimização de custos.
- Melhor alocação de recursos.

### Limitações
- Modelo assume linearidade.
- Fatores externos (sazonalidade, emergências) não estão modelados.
- Necessidade de atualização com novos dados.
